In [7]:
import os

dxf_folder = r"C:\Users\rafael.scatolin\OneDrive - Orguel\ai_designer_database\dxf files\training"
dxf_files = [os.path.join(dxf_folder, file) for file in os.listdir(dxf_folder) if file.endswith('.dxf')]

print(f"Found {len(dxf_files)} DXF files. Processing...")

Found 30 DXF files. Processing...


In [ ]:
from typing import List, Dict
import win32com.client
from shapely.geometry import box

acad = win32com.client.Dispatch("AutoCAD.Application")
acad.Visible = False # optional: run in background

labels: List[Dict] = []
for file in dxf_files:
    doc = acad.Documents.Open(file)
    modelSpace = doc.ModelSpace
    
    for text in modelSpace:
        if text.ObjectName is not "AcDbText": continue
        
        textString = text.TextString
        font = text.StyleName
        height = text.Height
        rotation = text.Rotation
        insertion = text.InsertionPoint
        min_point, max_point = text.GetBoundingBox()
        polygon = box(min_point[0], min_point[1], max_point[0], max_point[1])
        
        for character in textString:
            labels.append(
            {
                "file": file,
                "char": character,
                "font": font,
                "height": height,
                "rotation": rotation,
                "insertion_x": insertion[0],
                "insertion_y": insertion[1],
                "bbox": polygon
            }
            )
    
    doc.Close(False)

In [ ]:
import pandas

# Convert to DataFrame
dataframe = pandas.Dataframe(labels)
dataframe.to_pickle("character_dataframe.pkl")
print(f"Labeled {len(df)} characters using AutoCAD COM.")